In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
ds = tfds.load('multi_news', split='train', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:59<00:00, 59.64s/ url]

Extraction completed...: 100%|██████████| 1/1 [01:15<00:00, 75.21s/ file]
Dl Size...: 245 MiB [01:15,  3.26 MiB/s]
Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Dataset multi_news downloaded and prepared to C:\Users\Brad\tensorflow_datasets\multi_news\1.0.0. Subsequent calls will reuse this data.


ValueError: Visualisation not supported for dataset `multi_news`